In [ ]:
import os
import sys
sys.path.append("..")
import canon

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm
from mpl_toolkits.axes_grid1 import Grid
from PIL import Image
%matplotlib inline

import tensorflow as tf
import keras, keras.layers as L, keras.backend as K

from canon.autoencode.feeder import ImageDataFeeder

IMAGE_DIR = os.path.join("img", "processed_981")
TEST_DIR = os.path.join("img", "test_981")
filenames = [filename for filename in os.listdir(IMAGE_DIR)]
IMAGE_SHAPE = (128, 128)
feeder = ImageDataFeeder(IMAGE_SHAPE, batch_size=30, training_dir=IMAGE_DIR, test_dir=TEST_DIR)
X_test = feeder.get_test_set()
print(X_test.shape, X_test.max(), X_test.min())

In [ ]:
import canon.autoencode
from canon.autoencode import reset_tf_session
from canon.autoencode.models import load_encoder, load_decoder

s = reset_tf_session()
model_name = "ae_conv_4_256"
encoder = load_encoder(model_name)
decoder = load_decoder(model_name)
encoder.summary()
decoder.summary()

inp = L.Input(IMAGE_SHAPE)
code = encoder(inp)
reconstruction = decoder(code)
autoencoder = keras.models.Model(inputs=inp, outputs=reconstruction)
autoencoder.compile(optimizer="adadelta", loss='binary_crossentropy')
reconstruction_mse = autoencoder.evaluate(X_test, X_test, verbose=1)
print("Convolutional Autoencoder Binary CrossEntropy:", reconstruction_mse)

In [ ]:
fig, ax = plt.subplots(5, 3, figsize=(6, 10))
for i, idx in enumerate(np.random.choice(len(X_test), 5)):
    img = X_test[idx]
    code = encoder.predict(img[None])[0]  # img[None] is the same as img[np.newaxis, :]
    reco = decoder.predict(code[None])[0]

    ax0 = ax[i, 0]
    ax0.imshow(img, cmap='gray')
    ax0.axis('off')

    ax1 = ax[i, 1] 
    if np.prod(code.shape) == 32:
        code_shape = (8, 4)
    elif np.prod(code.shape) == 128:
        code_shape = (16, 8)
    else:
        code_shape = [int(np.ceil(np.sqrt(code.shape[-1]))),-1]
    ax1.imshow(code.reshape(code_shape), cmap='gray')
    ax1.axis('off')

    ax2 = ax[i, 2] 
    # plt.title("Reconstructed")
    ax2.imshow(reco, cmap='gray')
    ax2.axis('off')
    
    ax0.text(0.85, 0.9, "%d" % idx, color='w', fontsize=10, transform=ax0.transAxes)
    if i == 0:
        ax0.set_title(r'Original', fontsize=12)
        ax1.set_title(r'Code', fontsize=12)
        ax2.set_title(r'Reconstructed', fontsize=12)

fig.subplots_adjust(bottom=0, top=0.9, left=0, right=0.9, wspace=0.02, hspace=0.02)

In [ ]:
# image 96, 25, 99, 49, 64, 2

fig, ax = plt.subplots(5, 3, figsize=(6, 10))
for i, idx in enumerate([96, 25, 99, 49, 64, 2][:5]):
    img = X_test[idx]
    code = encoder.predict(img[None])[0]  # img[None] is the same as img[np.newaxis, :]
    reco = decoder.predict(code[None])[0]

    ax0 = ax[i, 0]
    ax0.imshow(img, cmap='gray')
    ax0.axis('off')

    ax1 = ax[i, 1] 
    if np.prod(code.shape) == 32:
        code_shape = (8, 4)
    elif np.prod(code.shape) == 128:
        code_shape = (16, 8)
    else:
        code_shape = [int(np.ceil(np.sqrt(code.shape[-1]))),-1]
    ax1.imshow(code.reshape(code_shape), cmap='gray')
    ax1.axis('off')

    ax2 = ax[i, 2] 
    ax2.imshow(reco, cmap='gray')
    ax2.axis('off')
    
    if i == 0:
        ax0.set_title(r'Original', fontsize=12)
        ax1.set_title(r'Latent Features', fontsize=12)
        ax2.set_title(r'Reconstructed', fontsize=12)

fig.subplots_adjust(bottom=0, top=0.9, left=0, right=0.9, wspace=0.02, hspace=0.02)
fig.savefig("img/review_model.pdf", bbox_inches='tight', dpi=300)